In [1]:
#conda install -c conda-forge spacy-model-en_core_web_sm
import re
import pandas as pd
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing


import logging

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
df =pd.read_csv('woundabstracts.csv')
##print(df['abstract'])
df=df.dropna().reset_index(drop=True)
##send=[row.split(' ') for row in df]
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"]) # disabling Named Entity Recognition for speed
def cleaning(doc):
    txt=[token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['abstract'])
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean = pd.DataFrame({'abstract': txt})
df_clean = df_clean.dropna().drop_duplicates()
print(df_clean.shape)
sent = [row.split() for row in df_clean['abstract']]
print(sent[:2])
import multiprocessing

from gensim.models import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=3,window=2,vector_size=50,sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20,workers=cores-1)

t = time()

w2v_model.build_vocab(sent, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()

w2v_model.train(sent, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

##w2v_model.init_sims(replace=True)
print(w2v_model.wv.most_similar(positive=["wound"]))
##w2v_model.wv.similarity("moe_'s", 'tavern')





Time to clean up everything: 0.0 mins
(8, 1)
[['wound', 'healing', 'importantabhd', 'physiological', 'process', 'maintain', 'integrity', 'skin', 'trauma', 'accident', 'intent', 'procedure', 'normal', 'wound', 'healing', 'involve', 'successive', 'overlap', 'phase', 'include', 'hemostasis', 'inflammatory', 'phase', 'proliferative', 'phase', 'remodeling', 'phase', 'aberration', 'wound', 'healing', 'excessive', 'wound', 'heal', 'hypertrophic', 'scar', 'keloid', 'chronic', 'wound', 'ulcer', 'impair', 'normal', 'physical', 'function', 'large', 'number', 'sophisticated', 'experimental', 'study', 'provide', 'insight', 'wound', 'healing', 'article', 'highlight', 'information', 'main', 'text', 'include', 'wound', 'healing', 'ii', 'wound', 'healing', 'fetus', 'adult', 'iii', 'prostaglandin', 'wound', 'healing', 'iv', 'pathogenesis', 'excessive', 'wound', 'healing', 'v', 'epidemiology', 'excessive', 'wound', 'healing', 'vi', 'vitro', 'vivo', 'study', 'excessive', 'wound', 'healing', 'vii', 'stem',

INFO - 00:53:52: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=50, alpha=0.03)', 'datetime': '2022-04-01T00:53:52.973751', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
INFO - 00:53:52: collecting all words and their counts
INFO - 00:53:52: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:53:52: collected 494 word types from a corpus of 1012 raw words and 8 sentences
INFO - 00:53:52: Creating a fresh vocabulary
INFO - 00:53:52: Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 70 unique words (14.17004048582996%% of original 494, drops 424)', 'datetime': '2022-04-01T00:53:52.984019', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'prepare_vocab'}
INFO - 00:53:52: Word2Vec lifecycle event {'msg': 'effective_min_

INFO - 00:53:53: worker thread finished; awaiting finish of 6 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 5 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 4 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 3 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 2 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 1 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 0 more threads
INFO - 00:53:53: EPOCH - 10 : training on 1012 raw words (41 effective words) took 0.0s, 4217 effective words/s


Time to build vocab: 0.0 mins


INFO - 00:53:53: worker thread finished; awaiting finish of 6 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 5 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 4 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 3 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 2 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 1 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 0 more threads
INFO - 00:53:53: EPOCH - 11 : training on 1012 raw words (33 effective words) took 0.0s, 2380 effective words/s
INFO - 00:53:53: worker thread finished; awaiting finish of 6 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 5 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 4 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 3 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 2 m

INFO - 00:53:53: worker thread finished; awaiting finish of 6 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 5 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 4 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 3 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 2 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 1 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 0 more threads
INFO - 00:53:53: EPOCH - 24 : training on 1012 raw words (28 effective words) took 0.0s, 2765 effective words/s
INFO - 00:53:53: worker thread finished; awaiting finish of 6 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 5 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 4 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 3 more threads
INFO - 00:53:53: worker thread finished; awaiting finish of 2 m

Time to train the model: 0.01 mins
[('factor', 0.9940826892852783), ('cell', 0.9909210801124573), ('amp', 0.9887978434562683), ('target', 0.9875363707542419), ('collagen', 0.9862659573554993), ('topical', 0.9848151206970215), ('effect', 0.9841198921203613), ('approach', 0.983677327632904), ('development', 0.9830446243286133), ('activity', 0.9822317957878113)]


In [2]:
print(w2v_model.wv.most_similar(positive=["wound"]))

[('factor', 0.9940826892852783), ('cell', 0.9909210801124573), ('amp', 0.9887978434562683), ('target', 0.9875363707542419), ('collagen', 0.9862659573554993), ('topical', 0.9848151206970215), ('effect', 0.9841198921203613), ('approach', 0.983677327632904), ('development', 0.9830446243286133), ('activity', 0.9822317957878113)]


In [3]:
print(w2v_model.wv.key_to_index)

w2v_model.wv.similarity("wound", 'healing')


{'wound': 0, 'healing': 1, 'factor': 2, 'growth': 3, 'cell': 4, 'process': 5, 'amp': 6, 'cytokine': 7, 'clinical': 8, 'tissue': 9, 'study': 10, 'stem': 11, 'review': 12, 'formulation': 13, 'involve': 14, 'therapy': 15, 'development': 16, 'topical': 17, 'include': 18, 'skin': 19, 'role': 20, 'molecular': 21, 'phase': 22, 'excessive': 23, 'matrix': 24, 'complex': 25, 'chronic': 26, 'family': 27, 'heal': 28, 'activity': 29, 'fibroblast': 30, 'gene': 31, 'repair': 32, 'ulcer': 33, 'focus': 34, 'endothelial': 35, 'pdgf': 36, 'abhd': 37, 'application': 38, 'collagen': 39, 'trial': 40, 'present': 41, 'current': 42, 'antimicrobial': 43, 'extracellular': 44, 'need': 45, 'function': 46, 'normal': 47, 'inflammatory': 48, 'proliferative': 49, 'peptide': 50, 'essential': 51, 'follow': 52, 'target': 53, 'article': 54, 'high': 55, 'provide': 56, 'address': 57, 'specific': 58, 'delivery': 59, 'new': 60, 'state': 61, 'regulate': 62, 'patient': 63, 'effect': 64, 'epidermal': 65, 'numerous': 66, 'platele

0.98168874

In [4]:
# read_file = pd.read_csv ('proteins.txt',sep='\n',header=None)
# read_file.columns = ['Name']
# read_file.to_csv ('protein.csv', index=None)


In [5]:
# read_file = pd.read_csv ('genes.txt',sep='\n',header=None)
# read_file.columns = ['Name']
# read_file.to_csv ('gene.csv', index=None)

In [6]:
proteins=pd.read_csv('protein.csv')
genes=pd.read_csv('gene.csv')

,Name
0,17betaHSD
1,ABHD10
2,ABHD11
3,ABRAXAS2
4,ACADL


In [14]:
brief_cleaning = (re.sub(" ", '', str(row)).lower() for row in genes['Name'])
genes=pd.DataFrame({'Name': brief_cleaning})
for index, row in genes. iterrows():
    try:
        #print(row['Name']+"--")
        print(w2v_model.wv.similarity("wound",row['Name'])," ",end='')
    except:
        continue
    print(row['Name'])

0.9594853  cytokine


In [15]:
brief_cleaning = (re.sub(" ", '', str(row)).lower() for row in proteins['Name'])
proteins=pd.DataFrame({'Name': brief_cleaning})
for index, row in proteins. iterrows():
    try:
        print(w2v_model.wv.similarity("wound",row['Name'])," ",end='')
    except:
        continue
    print(row['Name'])

0.9862659  collagen
0.9594853  cytokine
0.96762276  family


In [16]:
w2v_model.wv.similarity("cytokine", 'collagen')

0.94097775